In [2]:
import numpy as np
from tensorflow.keras.applications import vgg16

In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
#from tensorflow.keras.layers.normalization import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop

In [14]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import decode_predictions

In [29]:
train_data_dir = 'C:\\Users\\arjun\\Desktop\\Downs_Syndrome_Detection\\train'
test_data_dir = 'C:\\Users\\arjun\\Desktop\\Downs_Syndrome_Detection\\test'

In [28]:
from keras_preprocessing.image import ImageDataGenerator

In [15]:
img_rows, img_cols = 224, 224 
model = vgg16.VGG16(weights='imagenet', include_top = False, input_shape=(img_rows,img_cols,3))


ValueError: The input must have 3 channels; got `input_shape=(224, 224, 1)`

In [6]:
for layer in model.layers:
    layer.trainable = False

In [16]:
#no trainable paramters in our model since we gonna use the pre trained weights
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [12]:
def addTopModel(bottom_model, num_classes, D=256):
    #creates the top or head of the model that will be placed ontop of the bottom layers

    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D, activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model

In [18]:
num_classes=2
newmodel= addTopModel(model,num_classes)

In [26]:
modelnew = Model(inputs = model.input, outputs = newmodel)


In [27]:
modelnew.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [30]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')

In [31]:
train_batchsize = 16

In [33]:
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical')

Found 720 images belonging to 2 classes.


In [36]:
modelnew.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

In [42]:
history = modelnew.fit_generator(
    train_generator,
    steps_per_epoch = 32,
    epochs = 5,
   )

Epoch 1/5
 7/32 [=====>........................] - ETA: 4:53 - loss: 6.5828 - accuracy: 0.5268

KeyboardInterrupt: 

In [43]:
modelnew.save('facedetect.h5')